In [ ]:
# nbi:hide_in
##########################
# Created on Mar 2020
# @author: juans
##########################

In [9]:
# nbi:hide_in
# Requried libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive
%matplotlib inline

In [10]:
# nbi:hide_in
# Functions to create different types of quantizers
def mrQuantizeUniform(aNum, nBits):
    #Uniformly mid rise quantize signed fraction aNum with nBits
    #Notes:The overload level of the quantizer should be 1.0
        
    if (np.abs(aNum) >= 1):
        code = 2**(nBits-1)-1
    else:
        code = int(2**(nBits-1) * np.abs(aNum))
        
    if(aNum >= 0):
       return int(code)
    else:
       return int(code + 2**(nBits-1))
            
def mrDequantizeUniform(aQuantizedNum, nBits):
    #Uniformly mid rise dequantizes nBits-long number aQuantizedNum 
    #into a signed fraction
    if (aQuantizedNum>>(nBits-1)):
        sign = -1
    else:
        sign = 1
            
    if (sign == -1):
        aNum = sign * float(np.abs(aQuantizedNum - 2**(nBits-1) + 0.5))/(2**(nBits-1))
    else:
        aNum = sign * float(np.abs(aQuantizedNum) + 0.5)/(2**(nBits-1))
        
    return aNum

def mtQuantizeUniform(aNum,nBits):
    #Uniformly quantize signed fraction aNum with nBits
    #Notes:The overload level of the quantizer should be 1.0    
    if(aNum >= 0):
        s = 0
    else:
        s = 1
        
    if(np.abs(aNum) >= 1):
        code = 2**(nBits-1)-1
    else:
        code = int(((2**(nBits)-1)*np.abs(aNum)+1)/2)
        
        
    if(s == 0):
       return int(code)
    else:
       return int(code + 2**(nBits-1))
   
def mtDequantizeUniform(aQuantizedNum,nBits):
    #Uniformly dequantizes nBits-long number aQuantizedNum 
    #into a signed fraction
    if (aQuantizedNum>>(nBits-1)):
        sign = -1
    else:
        sign = 1
    
    if (sign == -1):
        aNum =  sign*float(2*np.abs(aQuantizedNum - 2**(nBits-1)))/(2**nBits -1)    
    else:
        aNum =  sign*float(2*np.abs(aQuantizedNum))/(2**nBits -1) 

    return aNum

In [11]:
# nbi:hide_in
# Interactive plot generation to exemplify different quantizers
def plot(nBits, qType):
    
    # Create a vector of amplitudes with at least 4 points per
    # Quantization step
    x = np.linspace(-1, 1, 2**nBits * 4)
    # Array for Quantized codes
    y = np.zeros_like(x, dtype=int)
    # Array for Dequantized float values
    z = np.zeros_like(x)
    
    # Select between MidTread and MidRise
    # For each quantize and dequantize each value
    # NOTE: because this is a non vectorized quantization algo
    # it is no particularly fast. If used you SHOULD vectorize it
    if qType == 'MidTread':
        for i in np.arange(len(y)):
            y[i] = mtQuantizeUniform(x[i], nBits)
            z[i] = mtDequantizeUniform(y[i], nBits)
    else:
        for i in np.arange(len(y)):
            y[i] = mrQuantizeUniform(x[i], nBits)
            z[i] = mrDequantizeUniform(y[i], nBits)

    plt.plot(x, z, '.')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([-1, 1])
    plt.ylim([-1, 1])
    plt.xticks(plt.yticks()[0], rotation=90)
    plt.grid(True)
    plt.xlabel('Continuous Amplitude')
    plt.ylabel('Quantized Amplitude')
    plt.title('Uniform Quantizer')
    plt.tight_layout()
    plt.show();

# Execute interaction
interactive_plot = interactive(plot, nBits=(1, 4 , 1), qType=['MidTread', 'MidRise'])
interactive_plot

interactive(children=(IntSlider(value=2, description='nBits', max=4, min=1), Dropdown(description='qType', opt…